In [ ]:
# pip install pingouin

In [90]:
# import necessary packages

import pandas as pd
import itertools
import numpy as np
# import scipy.stats
from statsmodels.stats.weightstats import ztest as ztest
import time

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import networkx as nx
from random import randint

import math
# import pingouin as pg

In [126]:
# import necessary data

result = pd.read_csv("/Users/robinkong/codes/result.csv")
# paperdata = pd.read_csv("/Users/robinkong/codes/paperdata.csv")
# cb = pd.read_csv("/Users/robinkong/codes/citationBara.csv")
# primdata = pd.read_csv("/Users/robinkong/codes/primdata.csv")
# pair = pd.read_csv("/Users/robinkong/codes/pairs.csv", chunksize = 1000000)
# mfpairs = pd.read_csv("/Users/robinkong/codes/mfpairs.csv")
# sim_mf = pd.read_csv("/Users/robinkong/codes/mfpairs_similarity.csv")

In [140]:
# DataFrame: {citing_doi, citing_year, cited_doi}

citinfo = primdata[["doi", "year"]].drop_duplicates(subset = 'doi')
cols = ['citing_doi', 'citing_year', 'cited_doi']

citinfo = citinfo.merge(cb, left_on = 'doi', right_on = 'citing_doi',
              how = 'inner').drop(['doi'], axis=1).rename(
                  {'year': 'citing_year'}, axis=1).reindex(
                      columns=cols).sort_values('cited_doi')

citinfo.head()

,citing_doi,citing_year,cited_doi
120140,10.1103/PhysRev.2.109,1913,10.1103/PhysRev.1.124
123113,10.1103/PhysRev.40.281,1932,10.1103/PhysRev.1.124
120191,10.1103/PhysRev.21.250,1923,10.1103/PhysRev.1.124
144119,10.1103/PhysRev.8.738,1916,10.1103/PhysRev.1.124
2740023,10.1103/PhysRevSeriesI.2.1,1894,10.1103/PhysRev.1.2


## Basic Statistics

In [144]:
print("Total papers: ", result.doi.nunique())
print("Papers with gender info: ", paperdata.doi.nunique())
print("Papers with primary gender info: ", primdata.doi.nunique())

Total papers:  541448
Papers with gender info:  375738
Papers with primary gender info:  273020


In [145]:
# journal query condition

s = "PRL"

print(f"Total {s} papers: ", result[result['journal'] == s].doi.nunique())
print(f"{s} papers with gender info: ", paperdata[paperdata['journal'] == s].doi.nunique())

df = primdata[primdata['journal'] == s]

print(f"{s} papers with primary gender info: ", df.doi.nunique())
print(f"{s} papers written primarily by men: ", len(df[df['gender']=='male']))
print(f"{s} papers written primarily by women: ", len(df[df['gender']=='female']))

Total PRL papers:  110080
PRL papers with gender info:  76527
PRL papers with primary gender info:  54460
PRL papers written primarily by men:  48950
PRL papers written primarily by women:  5510


In [156]:
# PACS query condition

pacs = '90'

print(f"Total PACS{pacs} papers: ", result[result[f'pacs{pacs}'] == 1].doi.nunique())
print(f"PACS{pacs} papers with gender info: ", paperdata[paperdata[f'pacs{pacs}'] == 1].doi.nunique())

df = primdata[primdata[f'pacs{pacs}'] == 1]

print(f"PACS{pacs} papers with primary gender info: ", df.doi.nunique())
print(f"PACS{pacs} papers written primarily by men: ", len(df[df['gender']=='male']))
print(f"PACS{pacs} papers written primarily by women: ", len(df[df['gender']=='female']))

Total PACS90 papers:  26601
PACS90 papers with gender info:  12628
PACS90 papers with primary gender info:  9597
PACS90 papers written primarily by men:  8469
PACS90 papers written primarily by women:  1128


## Preparing m-f similar pairs

In [7]:
# dealing with massive pair dataFrame

pair = list(pair)

In [29]:
mfpairs = pd.DataFrame(columns=list(pair[0].columns))

In [33]:
for p in tqdm(pair):
    mfpairs = pd.concat([mfpairs, p[p['gender1'] != p['gender2']]])

100%|██████████| 69/69 [31:34<00:00, 27.46s/it]


In [58]:
mfpairs = mfpairs.drop_duplicates().reset_index(drop=True)
print(len(mfpairs))

mfpairs.to_csv(f'/Users/robinkong/codes/mfpairs.csv',index=False)

14490273


In [59]:
len(mfpairs[mfpairs['qval']<0.002])/len(x)

0.10173576439864176

## Counting for each similar pair

In [5]:
sim_mf = mfpairs[['paper1', 'paper2', 'gender1', 'gender2', 'year1', 'year2', 'keyval', 'qval']]

In [ ]:
# Create a dummy data (test1, test2) to perform SQL:
# Create a DataFrame that treats citation info for male and female papers
test1 = sim_mf.merge(citinfo, left_on = 'paper1',
                    right_on = 'cited_doi', how = 'left').drop(
                        ["cited_doi", "citing_year"], axis=1).drop_duplicates(
                            subset=['paper1', 'citing_doi'])
test1['count'] = test1.groupby(['paper1'])['paper1'].transform('count')
test1 = test1.drop(["citing_doi"], axis=1).rename({'count': 'count1'}, axis=1)

test1 = test1.drop_duplicates(subset=['paper1', 'paper2'])[['paper1', 'count1']]
sim_mf = sim_mf.merge(test1, on='paper1', how='inner')

test2 = sim_mf.merge(citinfo, left_on = 'paper2',
                    right_on = 'cited_doi', how = 'left').drop(
                        ["cited_doi", "citing_year"], axis=1).drop_duplicates(
                            subset=['paper2', 'citing_doi'])
test2['count'] = test2.groupby(['paper2'])['paper2'].transform('count')
test2 = test2.drop(["citing_doi"], axis=1).rename(
    {'count': 'count2'}, axis=1).drop_duplicates(
        subset=['paper1', 'paper2'])[['paper2', 'count2']]
sim_mf = sim_mf.merge(test2, on='paper2', how='inner')

# Delete the dummy data
del test1
del test2

In [22]:
# Calculate year & centrality difference among pairs

listd = []
listy = []

for c in range(len(sim_mf)):
    if sim_mf['gender1'][c] == 'male':
        listd.append(sim_mf['count1'][c] - sim_mf['count2'][c])
        listy.append(sim_mf['year1'][c] - sim_mf['year2'][c])
    else:
        listd.append(sim_mf['count2'][c] - sim_mf['count1'][c])
        listy.append(sim_mf['year2'][c] - sim_mf['year1'][c])

sim_mf['countd'] = listd
sim_mf['yeard'] = listy

# Keep the minimum q-value per pair
sim_mf = sim_mf.sort_values('qval').drop_duplicates(
    subset=['paper1', 'paper2'], keep='first')

100%|██████████| 14490273/14490273 [10:25<00:00, 23159.47it/s]


In [38]:
sim_mf.to_csv("/Users/robinkong/codes/mfpairs_similarity.csv", index=False)

## Similarity Analysis

In [76]:
# filter conditions

# sdoi = primdata[primdata['journal'] == "PRL"].doi
# sdoi = primdata[primdata['pacs90'] == 1].doi

df = sim_mf.copy()

# df = df[df['paper1'].isin(sdoi)]
# df = df[df['paper2'].isin(sdoi)].reset_index(drop=True)

In [ ]:
# filter year constraints

N = 5

df = df[df['yeard'] >= -N]
df = df[df['yeard'] <= N].reset_index(drop=True)

In [77]:
p = 0.001
df_vld = df[df['qval'] < p]

print("N: ", len(df))
print("M: ", len(df_vld))
print("M/N: ", len(df_vld)/len(df))

N:  2730100
M:  410418
M/N:  0.15033075711512398


In [78]:
# getting mlist and fmlist (count per gender)

mlist = []
fmlist = []

for i in tqdm(range(len(df_vld))):
    if df_vld['gender1'][i] == 'male':
        mlist.append(df_vld['count1'][i])
        fmlist.append(df_vld['count2'][i])
    else:
        mlist.append(df_vld['count2'][i])
        fmlist.append(df_vld['count1'][i])

# Find z-scores (for p-values)
ztest(mlist, fmlist, value=0)

100%|██████████| 410418/410418 [00:04<00:00, 95449.02it/s]


(19.150151095942956, 9.65063481804804e-82)

In [ ]:
# Find z-scores (for p-values)
print(ztest(mlist, fmlist, value=0))
print('')

# Find effect size
print(pg.compute_effsize(mlist, fmlist, paired=True, eftype='hedges'))